In [1]:
import requests
import pandas as pd
from env import api_key
from datetime import datetime

### /v8/finance/spark - historical data for various intervals and ranges

In [ ]:
# Grabbing 5 years of close prices for Microsoft and Apple
headers = {
    'x-api-key': api_key
    }

response = requests.request("GET", 'https://yfapi.net/v8/finance/spark?interval=1d&range=5y&symbols=AAPL,MSFT', headers=headers)

In [3]:
# Saving response to data 
data = response.json()

In [28]:
# Creating an empty DataFrame
df = pd.DataFrame()

# Looping through each symbol in data
for symbol in data:
    
    # Creating a series for timestamp and closing price
    timestamp = pd.Series(data[symbol]['timestamp'])
    price = pd.Series(data[symbol]['close'])
    
    # Converting timestamp to year/month/day format
    timestamp = timestamp.apply(lambda x : datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))
    
    # Saving series into a the empty DataFrame I created earlier
    df[symbol + '_price'] = price
    df[symbol + '_date'] = timestamp

In [29]:
df

,MSFT_price,MSFT_date,AAPL_price,AAPL_date
0,62.30,2017-01-04,29.005,2017-01-04
1,62.30,2017-01-05,29.153,2017-01-05
2,62.84,2017-01-06,29.478,2017-01-06
3,62.64,2017-01-09,29.747,2017-01-09
4,62.62,2017-01-10,29.778,2017-01-10
...,...,...,...,...
1254,341.25,2021-12-28,179.290,2021-12-28
1255,341.95,2021-12-29,179.380,2021-12-29
1256,339.32,2021-12-30,178.200,2021-12-30
1257,336.32,2021-12-31,177.570,2021-12-31
